In [2]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import LAParams, LTTextBox, LTAnno, LTTextLine
from pdfminer.converter import PDFPageAggregator
import pdfminer
import pandas as pd
from tabula import wrapper

In [4]:
    def getValueByKey(self, pages, interpreter, device, keyWord):
        for page in pages:
            interpreter.process_page(page)
            layout = device.get_result()
            for x in layout:
                    if isinstance(x, LTTextBox) or isinstance(x, LTTextLine):
                    
                        rowIndex = 0
                        columnIndex = 0
                        for y in x:
                            if(y.get_text() == keyWord + '\n'):
                                rowIndex = y.bbox[0]
                                columnIndex = y.bbox[1]
                                break
                        
                        if(rowIndex != 0 and columnIndex != 0):
                            for x1 in layout:
                                if isinstance(x1, LTTextBox) or isinstance(x1, LTTextLine):
                                    for y1 in x1:
                                        if(y1.bbox[1] == columnIndex and y1.bbox[0] != rowIndex):
                                            return y1.get_text().strip()

    def getValueBySplit(self, pages, interpreter, device, keyWord):
        for page in pages:
            interpreter.process_page(page)
            layout = device.get_result()
            for x in layout:
                 if isinstance(x, LTTextBox) or isinstance(x, LTTextLine):
                        for y in x:
                            if (keyWord in y.get_text()):
                                return y.get_text().strip('\n').split(keyWord)[1]
                            
    def getAverageByLocation(self, pages, interpreter,device, keyWord):
#         print(dir(pages))
        format_years = []
        format_semesters = []
        format_sams = []
        format_aams = []
        n = 0
        for page in pages:
            interpreter.process_page(page)
            layout = device.get_result()
            count = 0
            if n == 1:
                for x in layout:
                    if isinstance(x, LTTextBox) or isinstance(x, LTTextLine):
#                         print(x.get_text())
                        if count == 0:
                            years = x.get_text().split('\n')

                            for year in years:
                                if(year != 'Year' and year != ''):
                                    format_years.append(year)
#                             print(format_years)
                            count += 1
                        elif count == 1:
                            semesters = x.get_text().split('\n')
                            for semester in semesters:
                                if(semester != 'Session' and semester != ''):
                                    format_semesters.append(semester)
                            count += 1
                        elif count == 2:
                            sams = x.get_text().split('\n')
                            for sam in sams:
                                if(sam != ''):
                                    format_sams.append(sam.replace('\n', ''))
                            count += 1
                        elif count == 3:
                            aams = x.get_text().split('\n')
                            for aam in aams:
                                if(aam != ''):
                                    format_aams.append(aam.replace('\n', ''))
                            count += 1
                a = pd.DataFrame({'Year': format_years, 'Session': format_semesters, 'Semester average mark': format_sams, 'Annual average mark': format_aams})
                return a
            n += 1
        
            for x in layout:
                if isinstance(x, LTTextBox) or isinstance(x, LTTextLine):

                    rowIndex = 0
                    columnIndex = 0
                    for y in x:
                        if(y.get_text() == keyWord + '\n'):
                            rowIndex = y.bbox[0]
                            columnIndex = y.bbox[1]
                            break

                    if(rowIndex != 0 and columnIndex != 0):
                        format_years = []
                        format_semesters = []
                        format_sams = []
                        format_aams = []
                        index = 0

                        for x1 in layout:
                            if isinstance(x1, LTTextBox) or isinstance(x1, LTTextLine):
                                if(x1.bbox[1] < columnIndex):
#                                     print(x1.get_text())
                                    if(index == 0):
                                        years = x1.get_text().split('\n')
                                        for year in years:
                                            if(year != 'Year' and year != ''):
                                                format_years.append(year)
                        #page1
#                                     if(index == 1):
#                                         semesters = x1.get_text().split('\n')
#                                         for semester in semesters:
#                                             if(semester != 'Session' and semester != ''):
#                                                 format_semesters.append(semester)
                                                    
#                                     if(index == 4):
#                                         sams = x1.get_text().split('\n')
#                                         for sam in sams:
#                                             if(sam != ''):
#                                                 format_sams.append(sam.replace('\n', ''))
                                                
#                                     if(index == 5):
#                                         aams = x1.get_text().split('\n')
#                                         for aam in aams:
#                                             if(aam != ''):
#                                                 format_aams.append(aam.replace('\n', ''))
                         #page2
                                    if(index == 2):
                                        semesters = x1.get_text().split('\n')
                                        for semester in semesters:
                                            if(semester != 'Session' and semester != ''):
                                                format_semesters.append(semester)

                                    if(index == 5):
                                        sams = x1.get_text().split('\n')
                                        for sam in sams:
                                            if(sam != ''):
                                                format_sams.append(sam.replace('\n', ''))

                                    if(index == 6):
                                        aams = x1.get_text().split('\n')
                                        for aam in aams:
                                            if(aam != ''):
                                                format_aams.append(aam.replace('\n', ''))

                                    index = index + 1

        a = pd.DataFrame({'Year': format_years, 'Session': format_semesters, 'Semester average mark': format_sams, 'Annual average mark': format_aams})
        return a

In [5]:
import pandas as pd
from tabula import wrapper

path = 'DemoTranscript.pdf'

file = open(path, 'rb')
praser = PDFParser(file)
doc = PDFDocument(praser, password='')
if not doc.is_extractable:
    raise PDFTextExtractionNotAllowe
rsrcmgr = PDFResourceManager(caching = False)
laparams = LAParams()
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)


In [6]:
Student_name = []
Student_name.append(pdf2TxtManager.getValueBySplit(PDFPage.create_pages(doc), interpreter, device, 'Student name '))
Student_id = []
Student_id.append(pdf2TxtManager.getValueBySplit(PDFPage.create_pages(doc), interpreter, device, 'Student ID '))
Student_course_id = []
Student_course_id.append(pdf2TxtManager.getValueBySplit(PDFPage.create_pages(doc), interpreter, device, 'Student course ID '))
student = [Student_name + Student_id + Student_course_id]

student_info = pd.DataFrame(columns = ('Studnet name', 'Student ID', 'Student course ID'), data = student)
# student_info = pd.DataFrame({'Studnet name': Student_name, 'Student ID': Student_id, 'Student course ID': Student_course_id})
        
print("-------This student information-------")
student_info
# student_info.sort_index(axis=1, ascending=False)



-------This student information-------


,Studnet name,Student ID,Student course ID
0,Jeremiah Simmons,103059683,103059683/1


In [7]:
value_course = []
value_course.append(pdf2TxtManager.getValueByKey(PDFPage.create_pages(doc), interpreter, device, 'Course'))
value_major = []
value_major.append(pdf2TxtManager.getValueByKey(PDFPage.create_pages(doc), interpreter, device, 'Major(s)'))
Credit_points_obtained = []
Credit_points_obtained.append(pdf2TxtManager.getValueBySplit(PDFPage.create_pages(doc), interpreter, device, 'Credit points obtained '))
courseDetails = [value_course + value_major + Credit_points_obtained]

course_details = pd.DataFrame(columns = ('Course', 'Major(s)', 'Credit points obtained'), data = courseDetails)
# course_details = pd.DataFrame(columns = {'Course', 'Major(s)', 'Credit points obtained'}, data = courseDetails)
# course_details = pd.DataFrame({'Course': value_course, 'Credit points obtained': Credit_points_obtained, 'Major(s)': value_major})

print("\n-------This course details-------")
course_details


-------This course details-------


,Course,Major(s),Credit points obtained
0,Bachelor of Advanced Computing,Computer Science,48


In [8]:
df = wrapper.read_pdf('DemoTranscript.pdf')
df

,Year,Session,Unit of study code,Unit of study name,Mark,Grade,Credit points
0,2016,S1C,ENGG1111,Integrated Engineering 1,76.0,DI,6
1,2016,S1C,INFO1103,Introduction to Programming,80.0,DI,6
2,2016,S1C,MATH1001,Differential Calculus,74.0,CR,3
3,2016,S1C,MATH1002,Linear Algebra,79.0,DI,3
4,2016,S1C,MKTG1001,Marketing Principles,73.0,CR,6
5,2016,S2C,ELEC1601,Foundations of Computer Systems,82.0,DI,6
6,2016,S2C,INFO1003,Foundations of Information Technology,84.0,DI,6
7,2016,S2C,INFO1105,Data Structures,86.0,HD,6
8,2016,S2C,MATH1003,Integral Calculus and Modelling,73.0,CR,3
9,2016,S2C,MATH1005,Statistics,84.0,DI,3


In [9]:



# list = pdf2TxtManager.getAverageByLocation(PDFPage.create_pages(doc), interpreter, device, 'Your Average Marks')
# print("\n-------This average marks-------")
# list.sort_index(axis=1, ascending=False)
list = pdf2TxtManager.getAverageByLocation(PDFPage.create_pages(doc), interpreter, device, 'Your Average Marks')
print("\n-------This average marks-------")
list.sort_index(axis=1, ascending=False)



-------This average marks-------


,Year,Session,Semester average mark,Annual average mark
0,2016,Semester 1,76.4,NA
1,2016,Semester 2,82.6,79.5


In [12]:
path = 'Demotranscript.pdf'

file = open(path, 'rb')
praser = PDFParser(file)
doc = PDFDocument(praser, password='')
if not doc.is_extractable:
    raise PDFTextExtractionNotAllowe
rsrcmgr = PDFResourceManager(caching = False)
laparams = LAParams()
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)
pages = PDFPage.create_pages(doc)

text  = []
y_axis = []
x_axis = []

data = pd.DataFrame(columns = ('y-axis', 'x-axis', 'text'))
for page in pages:
    interpreter.process_page(page)
    layout = device.get_result()
    for x in layout:
            if isinstance(x, LTTextBox) or isinstance(x, LTTextLine):
                for y in x:
                    text.append(y.get_text())
                    y_axis.append(int(y.bbox[1]))
                    x_axis.append(round(y.bbox[0], 2))

data['y-axis'] = y_axis
data['text'] = text
data['x-axis'] = x_axis

# data = data.sort_values(by = 'y-axis')

student_detail = pd.DataFrame(columns = ('Student name', 'Student ID', 'Student course ID'))
course_detail = pd.DataFrame(columns = ('Course', 'Credit points obtained', 'Major(s)'))
average_mark = pd.DataFrame(columns = ('Year', 'Session', 'Semester average mark', 'Annual average mark'))

course_row = 0
course_column = 0
major_row = 0
major = column = 0

for index, row in data.iterrows():
    if 'Student name' in row['text']:
        student_detail['Student name'] = [row['text'].strip('\n').split('Student name ')[1]]
    if 'Student ID' in row['text']:
        student_detail['Student ID'] = [row['text'].strip('\n').split('Student ID ')[1]]
    if 'Student course ID' in row['text']:
        student_detail['Student course ID'] = [row['text'].strip('\n').split('Student course ID ')[1]]
    if 'Credit points obtained' in row['text']:
        course_detail['Credit points obtained'] = [row['text'].strip('\n').split('Credit points obtained ')[1]]

        

def getCourseDetail(data, keyWord, course_detail):
    rowIndex = 0
    columnIndex = 0
    keyWord_1 = keyWord + '\n'
    
    for index, row in data.iterrows():
        if keyWord_1 in row['text']:
            columnIndex = row['y-axis']
            rowIndex = row['x-axis']

    for index, row in data.iterrows():
        if row['y-axis'] == columnIndex and row['x-axis'] != rowIndex:
            course_detail[keyWord] = [row['text'].strip('\n')]
            
getCourseDetail(data, 'Course', course_detail)
getCourseDetail(data, 'Major(s)', course_detail)

def getAverageMark(data, keyWord, average_mark):
    index = 0
    columnIndex = 0
    for index, row in data.iterrows():
        if keyWord in row['text']:
            columnIndex = row['y-axis']
    
    for index, row in data.iterrows():
        if row['y-axis'] < columnIndex and 'Year' in row['text']:
            index += 1
            break
        elif row['y-axis'] < columnIndex and index < 2:
            average_mark['Year'] = [row['text'].strip('\n')]
        elif row['y-axis'] < columnIndex and 'Session' in row['text']:
            index += 1
            break
        elif row['y-axis'] < columnIndex and index < 3:
            average_mark['Session'] = [row['text'].strip('\n')] 
        elif row['y-axis'] < columnIndex and 'Semester average mark' in row['text']:
            index += 1
            break        
        elif row['y-axis'] < columnIndex and index < 4:
            average_mark['Session'] = [row['text'].strip('\n')]         
            
            

getAverageMark(data, 'Your Average Marks', average_mark)       
        


data



,y-axis,x-axis,text
0,707,34.44,Course result history\n
1,671,34.44,The details listed below are for the course Ba...
2,658,34.44,a copy of your online result history and not a...
3,639,34.44,Student name Jeremiah Simmons\n
4,626,34.44,Student ID 103059683\n
5,614,34.44,Student course ID 103059683/1\n
6,587,34.44,Course details\n
7,639,456.12,Printed on: 22/Aug/2019\n
8,563,38.28,Course\n
9,548,38.28,Credit points obtained 48\n


In [13]:
student_detail

,Student name,Student ID,Student course ID
0,Jeremiah Simmons,103059683,103059683/1


In [14]:
course_detail

,Course,Credit points obtained,Major(s)
0,Bachelor of Advanced Computing,48,Computer Science
